In [ ]:
# Import necessary libraries
import win32com.client
import os
from zipfile import ZipFile
from datetime import date, timedelta

# Initialize the Outlook application interface
ol = win32com.client.Dispatch('Outlook.Application').GetNamespace("MAPI")

# Access the specific Outlook folde
folder = ol.Folders('WRO Reporting')
inbox = folder.Folders('Inbox')
msg = inbox.Items

# Calculate dates for filtering emails
today = date.today()
yesterday = date.today() - timedelta(days = 1)

# Adjust the date format for the email subject based on whether today is Monday or not
if date.today().strftime('%A') == "Monday":
    yesterday_date = (date.today() - timedelta(days=3)).strftime("%#m/%#d/%y")
else:
    yesterday_date = (date.today() - timedelta(days=1)).strftime("%#m/%#d/%y")
    
yesterday_to_zip = (date.today() - timedelta(days = 1)).strftime("%#m%#d%y")
date = today.strftime("%#m/%#d/%y")
print(date)

# Filter emails based on subject lines for daily and abandoned call reports
messages_daily = inbox.Items.Restrict(f"[Subject]='Wro Daily Report_New [{yesterday_date}] [07:05 PM]'")
messages_abadoned = inbox.Items.Restrict(f"[Subject]='Wro Daily Abandoned Calls Report_New [{yesterday_date}] [07:10 PM]'")
msgs_daily = messages_daily.GetLast()
msgs_abadoned = messages_abadoned.GetLast()

# Specify paths for saving attachments
daily_path = r'\\Lorem ipsum'
daily_folder_path = r'\\Lorem ipsum'
abadoned_path = r'\\Lorem ipsum'
abadoned_folder_path = r'\\Lorem ipsum'

# Process attachments in the daily report email
for attachment in msgs_daily.Attachments:
    # Save the attachment to a specified directory
    attachment.SaveASFile(os.path.join(daily_folder_path, attachment.FileName))
    print(f"attachment {attachment.FileName} from saved")
    file_name = attachment.FileName
    file_path = f'{daily_folder_path}\{file_name}'
    # Open and extract the ZIP file, then delete it
    with ZipFile(file_path, 'r') as zip:
        zip.printdir()
        print('Extracting all the files now...')
        zip.extractall(daily_folder_path)
        print('Done!')
    os.remove(file_path)
    
# Repeat the process for attachments in the abandoned calls report email
for attachment in msgs_abadoned.Attachments:
    attachment.SaveASFile(os.path.join(abadoned_folder_path, attachment.FileName))
    print(f"attachment {attachment.FileName} saved")
    file_name = attachment.FileName
    file_path = f'{abadoned_folder_path}\{file_name}'
    with ZipFile(file_path, 'r') as zip:
        zip.printdir()
        print('Extracting all the files now...')
        zip.extractall(abadoned_folder_path)
        print('Done!')
    os.remove(file_path)

In [ ]:
# Import necessary libraries
import pandas as pd
import xlwings as xl
import datetime as dt
import shutil

# Define paths to the server files
daily_server_path = r'\\Lorem ipsum'
abadoned_server_path = r'\\Lorem ipsum'
daily_server_path_new = r'\\Lorem ipsum'
abadoned_server_path_new = r'\\Lorem ipsum'

# Remove the original daily report file
os.remove(daily_server_path)

# Read the new daily report, skipping the first row
daily = pd.read_excel(daily_server_path_new, skiprows=[0])

# Initialize the xlwings app with Excel not visible
app = xl.App(visible=False)

# Open the new daily report Excel file
wb = xl.Book(daily_server_path_new)
ws = wb.sheets["Wro Daily Report_New"]

# Format the date and time columns
ws.range('D:D').number_format = '[$-en-US]dd/mm/yy hh:mm:ss AM/PM;@'
ws.range('E:E').number_format = '[$-en-US]dd/mm/yy hh:mm:ss AM/PM;@'

# Adjust the Call Start Time and Call End Time formats if they're datetime objects
df_temp = daily[["Call Start Time", "Call End Time"]].applymap(lambda x: str(x) if isinstance(x, dt.time) else x)

# Rename the sheet for consistency
wb.sheets["Wro Daily Report_New"].name = "Wro Daily report"

# Save the workbook and close it
ws.range('D2').options(index=False).value = df_temp
wb.save(daily_server_path)
wb.close()

os.remove(daily_server_path_new)

print("Cisco daily dates updated correctly")

# Repeat similar steps for the abandoned calls report

# Remove the new daily report file after saving changes to the original path
os.remove(abadoned_server_path)

# Read the new abandoned calls report, skipping the first row
abadoned = pd.read_excel(abadoned_server_path_new, skiprows=[0])

# Open the new abandoned report Excel file
wb = xl.Book(abadoned_server_path_new)
ws = wb.sheets["Wro Daily Abandoned Calls Repor"]

# Format the date and time columns
ws.range('A:A').number_format = '[$-en-US]dd/mm/yy hh:mm:ss AM/PM;@'
ws.range('I:I').number_format = '[$-en-US]dd/mm/yy hh:mm:ss AM/PM;@'

# Adjust the Call Start Time and Call Abandon Time formats if they're datetime objects
df_temp = abadoned[["Call Start Time", "Call Abandon Time"]].applymap(lambda x: str(x) if isinstance(x, dt.time) else x)

# Rename the sheet for consistency
wb.sheets["Wro Daily Abandoned Calls Repor"].name = "Wro Daily Abandoned Calls repor"

# Update the worksheet with the formatted data
ws.range('A2').options(index=False).value = df_temp.iloc[:,0]
ws.range('I2').options(index=False).value = df_temp.iloc[:,1]

# Save the workbook and close it
wb.save(abadoned_server_path)
wb.close()

# Quit the xlwings app
app.quit()

# Remove the new daily report file after saving changes to the original path
os.remove(abadoned_server_path_new)

print("Cisco abadoned dates updated correctly")


